In [1]:
import pandas as pd
import numpy as np
import re
import razdel
import pymorphy2
from sklearn.decomposition import PCA
import torch
import nltk
from navec import Navec
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.cluster import AgglomerativeClustering
from sklearn.neighbors import NearestCentroid
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import TfidfTransformer 
from sklearn.feature_extraction.text import CountVectorizer 
import warnings

warnings.filterwarnings("ignore")

nltk.download('stopwords')

/home/darinka/.local/share/virtualenvs/news-parser-IZMhYhw-/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
[nltk_data] Downloading package stopwords to
[nltk_data]     /home/darinka/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [255]:
buhru_data = pd.read_csv('../data/news_buhru.csv')
bankiru_data = pd.read_csv('../data/banki_ru.csv')
rbc_data = pd.read_csv('../data/news_rbc.csv')
rbc_data = pd.read_csv('../data/news_lentaru.csv')

data = pd.concat([buhru_data, bankiru_data, rbc_data])
data = data.set_index(pd.Index(range(len(data))))

In [256]:
data

,title,date,content,num_views,num_comments,tags,url
0,"Повышение лимитов для УСН, упрощенные правила ...",07.10.2022,Редакция БУХ.1С напоминает о самых позитивных ...,1549,0,проверки бизнеса;лицензирование;предельные дох...,NaN
1,Единый семинар 1С для бухгалтеров и руководите...,07.10.2022,12 октября 2022 в 10:00 по местному времени фи...,5098,0,Единый семинар;семинар по бухучету и налогообл...,NaN
2,Работодателей обязали сдавать СЗВ-ТД при приос...,07.10.2022,С 7 октября 2022 года вступили в силу поправки...,1726,1,СЗВ-ТД;мобилизация;трудовой договор;отчетность...,NaN
3,Вправе ли работодатель отправлять форму СТД-Р ...,07.10.2022,"Эксперты Роструда разъяснили, вправе ли работо...",834,0,увольнение по инициативе сотрудника;СТД-Р;прав...,NaN
4,Опубликована видеозапись лекции об использован...,07.10.2022,На странице 1С:Лектория опубликована видеозапи...,781,0,1С:Лекторий;1С:Предприятие 8;СЭДО;пособия;1С:З...,NaN
...,...,...,...,...,...,...,...
29098,Мишустин заявил о готовности России перейти к ...,04.07.2022,Россия должна отказаться от догоняющей модели ...,0,0,Госэкономика,NaN
29099,Оценены успехи программы импортозамещения в Ро...,04.07.2022,С 2010 по 2021 год в рамках стратегии импортоз...,0,0,Госэкономика,NaN
29100,В Сбере стала доступна комбо-ипотека,04.07.2022,Сбер запустил уникальное предложение на рынке ...,0,0,Бизнес,NaN
29101,Назван срок появления электромотоциклов Aurus,04.07.2022,Серийное производство электрических мотоциклов...,0,0,Рынки,NaN


In [14]:
morph = pymorphy2.MorphAnalyzer()

In [15]:
stop_eng = set(stopwords.words('english') + ['a', 'b', 'c', 'd', 'e', 'f', 'g', 
                                            'h', 'i', 'j', 'k', 'l', 'm', 'n', 
                                            'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z'])
stop_rus = set(stopwords.words('russian') + ['а', 'о', 'у', 'ы', 'э', 'я', 'е', 
                                            'ё', 'ю', 'и', 'б', 'в', 'г', 'д', 
                                            'й', 'ж', 'з', 'к', 'л', 'м', 'н', 
                                            'п', 'р', 'с', 'т', 'ф', 'х', 'ц', 'ч', 'ш', 'щ']) 
stop_words = stop_eng.union(stop_rus)

In [12]:
def parse_tokens(text: str) -> list:
    words = re.split(r"\[|, '|'|\]", text)
    return " ".join(words).strip().split("  ")


def tokenize_with_razdel(text: str) -> list:
    """
    Токенизация с помощью модуля razdel
    """
    return [token.text for token in razdel.tokenize(text)]


def lemmatize(tokens: list) -> str:
    words = [morph.parse(token)[0].normal_form for token in tokens]
    return " ".join([word for word in words if word not in stop_words and len(word) > 2])


def preprocessing_name(text: str) -> str:
    """
    Токенизация и лемматизация текста
    """
    if isinstance(text, str):
        return lemmatize(tokenize_with_razdel(clean_text(text)))
    return ""

def clean_text(text):
    text = re.sub('[^а-яa-zёА-ЯA-ZЁ-]+', ' ', text.strip())
    text = re.sub(' - |- | -', ' ', text.lower())
    return re.sub('\s+', ' ', text)

def preprocessing_names(names: list):
    return [preprocessing_name(clean_text(name)) for name in names]

In [262]:
data['title_tokens'] = data['title'].apply(preprocessing_name)

In [263]:
data['tags_tokens'] = data['tags'].apply(preprocessing_name)

In [264]:
data['content_tokens'] = data['content'].apply(preprocessing_name)

In [6]:
data['text_tokens'] = data[['title_tokens', 'tags_tokens', 'content_tokens']].apply(' '.join, axis=1)

TypeError: sequence item 2: expected str instance, float found

In [8]:
cv = CountVectorizer() 
word_count_vector = cv.fit_transform(data['text_tokens'])
tfidf_transformer = TfidfTransformer(smooth_idf=True,use_idf=True) 
tfidf_transformer.fit(word_count_vector)

count_vector = cv.transform(data['text_tokens']) 
tf_idf_vector = tfidf_transformer.transform(count_vector)
feature_names = cv.get_feature_names()

In [9]:
tfidf_vectors = list()
for i in data.index:
    doc_vec = tf_idf_vector[i]
    df = pd.DataFrame(doc_vec.T.todense(), index=feature_names, columns=["tfidf"])
    
    tfidf_vec = list()
    for w in data['text_tokens'][i].split():
        try:
            tfidf_vec.append(df.loc[w]['tfidf'])
        except KeyError:
            tfidf_vec.append(0)
    
    tfidf_vectors.append(tfidf_vec)

data['tfidf_vectors'] = np.array(tfidf_vectors)

In [20]:
path = 'navec_news_v1_1B_250K_300d_100q.tar'
navec = Navec.load(path)

In [17]:
def get_emb(row, tfidf):
    embds = list()
    if row is not None:
        for i, word in enumerate(row.split()):
            word = word.replace('ё', 'е')
            if word in navec:
                embds.append(navec[word] * tfidf[i])
    
    embds = np.array(embds)

    if len(embds):
        return embds.mean(axis=0)
    else:
        return np.zeros((300))

In [18]:
def str_list(text):
    return list(map(float, text[1:-1].split(', ')))

In [13]:
data['tfidf_vectors'] = data['tfidf_vectors'].apply(str_list)

In [15]:
data['text_vectors'] = data.apply(lambda x: get_emb(x['text_tokens'], x['tfidf_vectors']), axis=1) 

In [16]:
pca = PCA(n_components=50)
pca.fit(torch.tensor(data['text_vectors']))
reduced_vectors = pca.transform(torch.tensor(data['text_vectors']))

In [17]:
aggcluster = AgglomerativeClustering(n_clusters=20)
clusters_aggl_pca = aggcluster.fit_predict(torch.tensor(reduced_vectors))
data['clusters'] = clusters_aggl_pca

: 

: 

In [7]:
data = pd.read_csv('data.csv')
data[['title', 'clusters']]

,title,clusters
0,"Повышение лимитов для УСН, упрощенные правила ...",17
1,Единый семинар 1С для бухгалтеров и руководите...,20
2,Работодателей обязали сдавать СЗВ-ТД при приос...,22
3,Вправе ли работодатель отправлять форму СТД-Р ...,20
4,Опубликована видеозапись лекции об использован...,20
...,...,...
12235,Замена валюте: инвесторы наращивают объем торг...,14
12236,Обзор рынков от Банки.ру: доллар может уйти в ...,14
12237,"Эксперт объяснил, что будет с долларом и евро ...",14
12238,Несмотря на ставки. Россияне несут деньги в ба...,14


In [8]:
with open('buh.txt', 'r') as f:
    buh_keywords = f.readline()


In [9]:
with open('gendir.txt', 'r') as f:
    gen_dir = f.readline()

In [10]:
import pickle

with open("pca_reduce", 'rb') as f:
    pca = pickle.load(f)

In [21]:
clean_buh = preprocessing_name(buh_keywords)
vector_buh = get_emb(clean_buh, tfidf=np.ones(len(clean_buh.split())))
reduces_buh = pca.transform([vector_buh])

In [63]:
reduces_buh.tolist()

[[0.35427126439110423,
  1.4234676268248796,
  -0.30443778124814413,
  0.10850139430647113,
  0.29460299295374676]]

In [65]:
with open("vector_buh", "wb") as f:
    pickle.dump(reduces_buh, f)
with open("vector_gen", "wb") as f:
    pickle.dump(reduces_gen, f)

In [22]:
clean_gen = preprocessing_name(gen_dir)
vector_gen = get_emb(clean_gen, tfidf=np.ones(len(clean_gen.split())))
reduces_gen = pca.transform([vector_gen])

In [25]:
with open("reduced_vect", 'rb') as f:
    reduced_vectors = pickle.load(f)

In [26]:
clusters_aggl_pca = data['clusters']

In [27]:
clf = NearestCentroid()
clf.fit(reduced_vectors, clusters_aggl_pca)
centroids = clf.centroids_

In [70]:
with open("clusters", 'wb') as f:
    pickle.dump(clusters_aggl_pca, f)
clusters_aggl_pca

0        17
1        20
2        22
3        20
4        20
         ..
12235    14
12236    14
12237    14
12238    14
12239    17
Name: clusters, Length: 12240, dtype: int64

In [71]:
data

,title,date,content,num_views,num_comments,tags,url,title_tokens,tags_tokens,content_tokens,text_tokens,tfidf_vectors,text_vectors,clusters
0,"Повышение лимитов для УСН, упрощенные правила ...",07.10.2022,Редакция БУХ.1С напоминает о самых позитивных ...,1549,0,проверки бизнеса;лицензирование;предельные дох...,NaN,повышение лимит усн упрощённый правило алкогол...,проверка бизнес лицензирование предельный дохо...,редакция бух напоминать самый позитивный бухга...,повышение лимит усн упрощённый правило алкогол...,"[0.034183826452033654, 0.049865348984263126, 0...",[-9.08896234e-03 -9.75437928e-03 8.12493323e-...,17
1,Единый семинар 1С для бухгалтеров и руководите...,07.10.2022,12 октября 2022 в 10:00 по местному времени фи...,5098,0,Единый семинар;семинар по бухучету и налогообл...,NaN,единый семинар бухгалтер руководитель пройти о...,единый семинар семинар бухучёт налогообложение...,октябрь местный время фирма проводить единый с...,единый семинар бухгалтер руководитель пройти о...,"[0.20564659740162877, 0.6527226913014548, 0.22...",[ 0.00717525 -0.02597658 -0.01977311 0.009604...,20
2,Работодателей обязали сдавать СЗВ-ТД при приос...,07.10.2022,С 7 октября 2022 года вступили в силу поправки...,1726,1,СЗВ-ТД;мобилизация;трудовой договор;отчетность...,NaN,работодатель обязать сдавать сзв-тд приостанов...,сзв-тд мобилизация трудовой договор отчётность...,октябрь год вступить сила поправка закон персо...,работодатель обязать сдавать сзв-тд приостанов...,"[0.08164188435189614, 0.04329531247874227, 0.0...",[-0.00987843 -0.01810046 -0.00253365 -0.002707...,22
3,Вправе ли работодатель отправлять форму СТД-Р ...,07.10.2022,"Эксперты Роструда разъяснили, вправе ли работо...",834,0,увольнение по инициативе сотрудника;СТД-Р;прав...,NaN,вправе работодатель отправлять форма стд-р уво...,увольнение инициатива сотрудник стд-р право ра...,эксперт роструд разъяснить вправе работодатель...,вправе работодатель отправлять форма стд-р уво...,"[0.17602018810066877, 0.1948469657385272, 0.06...",[ 8.92193802e-03 -1.39221698e-02 9.15071089e-...,20
4,Опубликована видеозапись лекции об использован...,07.10.2022,На странице 1С:Лектория опубликована видеозапи...,781,0,1С:Лекторий;1С:Предприятие 8;СЭДО;пособия;1С:З...,NaN,опубликовать видеозапись лекция использование ...,лекторий предприятие сэдый пособие зарплата уп...,страница лекторий опубликовать видеозапись лек...,опубликовать видеозапись лекция использование ...,"[0.13143021901449714, 0.25992047771525134, 0.3...",[ 0.03399515 -0.02355277 -0.01062908 -0.033326...,20
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12235,Замена валюте: инвесторы наращивают объем торг...,03.08.2022,По итогам июля объем торгов драгоценными метал...,1287,0,NaN,https://www.banki.ru/news/lenta/?id=10970142,замена валюта инвестор наращивать объём торг з...,NaN,итог июль объём торг драгоценный металл москов...,замена валюта инвестор наращивать объём торг з...,"[0.054376429322576886, 0.09689863926813268, 0....",[-1.17929131e-02 -9.12638754e-03 -1.58270448e-...,14
12236,Обзор рынков от Банки.ру: доллар может уйти в ...,03.08.2022,Российский рынок акций может попытаться сегодн...,1556,0,NaN,https://www.banki.ru/news/lenta/?id=10970140,обзор рынок банк доллар мочь уйти ход сессия н...,NaN,российский рынок акция мочь попытаться сегодня...,обзор рынок банк доллар мочь уйти ход сессия н...,"[0.037877294787606486, 0.2199232684245734, 0.0...",[-1.56688672e-02 1.16692239e-03 -1.10665299e-...,14
12237,"Эксперт объяснил, что будет с долларом и евро ...",03.08.2022,"В случае отмены биржевых торгов, которая возмо...",18816,0,NaN,https://www.banki.ru/news/lenta/?id=10970139,эксперт объяснить доллар евро отмена биржевой ...,NaN,случай отмена биржевой торг который возможный ...,эксперт объяснить доллар евро отмена биржевой ...,"[0.09441118764664563, 0.041536799207628025, 0....",[-1.30954683e-02 5.50444797e-03 1.32974787e-...,14
12238,Несмотря на ставки. Россияне несут деньги в ба...,03.08.2022,Объем наличных в обращении вновь сокращае

In [28]:
buh_clust = np.argmax(np.array([cosine_similarity([centroids[i]], reduces_buh) for i in range(20)]))

In [29]:
buh_clust

7

In [30]:
gen_clust = np.argmax(np.array([cosine_similarity([centroids[i]], reduces_gen) for i in range(20)]))

In [31]:
gen_clust

2

In [57]:
data[clusters_aggl_pca == 1].title.head(20)

8712     Официальный курс доллара поднялся на 1 рубль,...
8819     Официальный курс доллара вырос на 6 рублей 51...
8878     Официальный курс доллара понизился на 3,38 ру...
8999     Официальный курс доллара повысился более чем ...
9056     Официальный курс доллара снизился на 1,81 руб...
9120     Биржевой курс доллара дошел до 110 рублей, кур...
9124     Официальный курс доллара повысился на 11,5 ру...
9179     Официальный курс доллара вырос на 8,51 рубля,...
9228     Официальный курс доллара понизился на 5,94 ру...
9269     Официальный курс доллара не изменился, евро —...
9422     Официальный курс доллара повысился на 10,27 р...
9455     Официальный курс доллара вырос на 4,29 рубля,...
9516     Официальный курс доллара понизился на 3,63 ру...
9620     Официальный курс доллара снизился на 1,56 руб...
9663     Официальный курс доллара снизился на 3,71 руб...
9717     Официальный курс доллара снизился на 3,43 руб...
9770     Официальный курс доллара снизился на 3,25 руб.

In [37]:
def get_recom(target: int, role_vector: np.array, place: int):
    rating = np.argsort(cosine_similarity(reduced_vectors[clusters_aggl_pca == target], role_vector).reshape(-1))
    rang = np.arange(len(rating))
    d = data.iloc[rang[rating == place][0]]
    print(f"title:\n{d.title}")
    print(f"content:\n{d.content}")

In [38]:
get_recom(buh_clust, reduces_buh, 0)

title:
Отсрочка от мобилизации, больничные для самозанятых и рост налогов, маткапитала и пособий в 2023 году - все новости прошедшей недели за 5 минут (26.09-02.10.2022)
content:
Власти запустили на Госуслугах сервис по подаче заявлений на предоставление отсрочки от мобилизации. Минфин объяснил, как изменятся налоги в 2023 году. Минтруд определил условия и параметры оплаты больничных самозанятым. Рассказываем о главных новостях прошедшей недели. Будьте в курсе событий в стране.


В 2023 году бюджету потребуются дополнительные источники доходов. Но дополнительное налогообложение коснется только экспортеров, а налоги для граждан меняться не будут.
С 1 февраля 2023 года будут увеличены максимальные и минимальные суммы пособий по временной нетрудоспособности и в связи с материнством. Единовременное пособие при рождении ребенка будет повышено до 23 011 рублей. Максимальная выплата по больничному составит в 2023 году: 83 204 рубля в месяц для граждан со страховым стажем 8 и более лет; 66 564

In [39]:
get_recom(buh_clust, reduces_buh, 1)

title:
Правительство установило, какие организации могут ограничивать доступ к своей отчетности в ГИР БО
content:
Правительство определило перечень организаций, которые могут ограничивать доступ к своей бухгалтерской отчетности, размещенной в Государственном информационном ресурсе бухгалтерской (финансовой) отчетности (ГИР БО).
Соответствующее постановление от 16.09.2022 № 1624 опубликовано на Официальном интернет-портале правовой информации.
Напомним, статья 18 закона о бухучете (от 06.12.2011 № 402-ФЗ) разрешает Правительству РФ определить случаи, в которых доступ к информации в ГИР БО может быть ограничен.
В связи с этим правительство установило, что доступ к информации в указанном ресурсе может быть ограничен в следующих случаях:
Также постановление содержит правила ограничения и возобновления доступа к информации, содержащейся в ГИР БО.
В частности, ограничение и возобновление доступа к информации в ГИР БО осуществляется на основании соответствующих заявлений, направленных самими 

In [40]:
get_recom(buh_clust, reduces_buh, 2)

title:
Продажа товаров своему работодателю не попадает под налог на профессиональный доход  
content:
Минфин уточнил, можно ли применять налог на профессиональный доход в отношении доходов, полученных от реализации товаров своему работодателю.
В своем письме от 24.08.2022 № 03-11-06/82525 ведомство отмечает, что объектом обложения НПД признаются доходы от реализации товаров (работ, услуг, имущественных прав).
При этом не признаются объектом обложения данным налогом доходы от оказания (выполнения) физическими лицами услуг (работ) по гражданско-правовым договорам, при условии, что заказчиками услуг (работ) выступают работодатели указанных физических лиц или лица, бывшие их работодателями менее 2 лет назад.
Таким образом, доходы, полученные от реализации товаров своему работодателю, не признаются объектом налогообложения НПД.
Следить за новостями удобно в нашем новостном Telegram-канале. Присоединяйтесь!


In [42]:
get_recom(buh_clust, reduces_buh, 100)

title:
ПФР вводит новые коды для заполнения формы СЗВ-СТАЖ в отношении мобилизованных работников
content:
ПФР подготовил поправки в правила заполнения сведений о страховом стаже работников (форма СЗВ-СТАЖ). Проект соответствующего постановления опубликован на Едином портале для размещения проектов НПА.
Напомним, сведения о страховом стаже работников представляются работодателями в ПФР по форме СЗВ-СТАЖ, утвержденной постановлением Правления ПФР от 06.12.2018 № 507п (с изменениями и дополнениями).
Теперь ПФР подготовил изменения в порядок заполнения формы СЗВ-СТАЖ, позволяющие указывать периоды приостановления и возобновления действия трудового договора с работником, в частности в связи с мобилизацией.
Также учтены изменения правил исчисления периодов работы, дающих право на досрочную пенсию, в части включения в стаж периодов профессионального обучения или дополнительного профессионального образования с отрывом от работы сотрудников, работающих в районах Крайнего Севера и приравненных к

In [43]:
get_recom(gen_clust, reduces_gen, 0)

title:
Опубликована видеозапись лекции о ФСБУ 25: аренда по договору в валюте и учет по ПБУ 18 в решениях "1С:ERP" и "1С:Комплексная автоматизация"
content:
На странице 1С:Лектория опубликована видеозапись лекции о ФСБУ 25: аренда по договору в валюте и учет по ПБУ 18 в решениях "1С:ERP" и "1С:Комплексная автоматизация". Видеозапись состоит из двух частей:
А. Фроленкова. ФСБУ 25. Аренда по договору в валюте и учет по ПБУ 18 в программах 1C:ERP и 1С:Комплексная автоматизация
С расписанием онлайн-лекций и мероприятий по 1С можно ознакомиться на нашем сайте.


In [44]:
get_recom(gen_clust, reduces_gen, 1)

title:
Росстат ввел новые формы отчетности для театров, музеев и цирков
content:
Росстат утвердил новые годовые формы отчетности для организаций сферы культуры. Соответствующие приказы и опубликованы на официальном сайте статистического ведомства.
Так, с отчетности за 2022 год приказ от 16.09.2022 № 641 вводит новые годовые формы:
Новая форма № 8-НК «Сведения о деятельности музея» введена приказом от 22.09.2022 № 647 также с отчетности за 2022 год.
Перечисленные формы отчетности организации сферы культуры должны ежегодно сдавать в Министерство культуры
Прежние аналогичные формы отчетности, утвержденные приказами Росстата от 18.10.2021 № 713 и от 26.09.2018 № 584, упраздняются.
Следить за новостями удобно в нашем новостном Telegram-канале. Присоединяйтесь!


In [45]:
get_recom(gen_clust, reduces_gen, 2)

title:
Для упрощенцев установят правило расчета налога при смене места нахождения организации 
content:
Минфин планирует уточнить в НК РФ порядок исчисления единого налога по УСН при смене места нахождения организации или места жительства ИП.
Соответствующий законопроект, подготовленный для выполнения положений основных направлений налоговой политики, опубликован на Едином портале для размещения проектов НПА.
В частности, законопроект предлагает дополнить пункт 2 статьи 34621 НК РФ нормой, согласно которой при изменении места нахождения организации (места жительства индивидуального предпринимателя) из одного субъекта РФ в другой субъект РФ налог (авансовые платежи по налогу) исчисляется по ставке, действующей по новому месту нахождения организации (месту жительства ИП).
Как пояснили в министерстве, данное уточнение необходимо в случае, когда одним из субъектов РФ установлены пониженные налоговые ставки для УСН.
Следить за новостями удобно в нашем новостном Telegram-канале. Присоединяйт

In [46]:
get_recom(gen_clust, reduces_gen, 3)

title:
ФНС введет новый формат электронной доверенности налогоплательщика 
content:
ФНС изменит формат электронной доверенности, подтверждающей полномочия представителя налогоплательщика (плательщика сбора, плательщика страховых взносов, налогового агента). Проект соответствующего приказа опубликован на Едином портале для размещения проектов НПА.
Напомним, действующий формат электронной доверенности утвержден приказом от 30.04.2021 № ЕД-7-26/445@.
Вместе с тем с 14 июля 2022 года вступили в силу обновленные правила подписи электронных документов от имени юридических лиц и индивидуальных предпринимателей по доверенности. Соответствующие поправки в статьи 17.2 и 17.3 закона об электронной подписи (от 06.04.2011 № 63-ФЗ) внес Федеральный закон от 14.07.2022 № 339-ФЗ.
В частности, новыми нормами предусмотрена возможность применения при подписи электронных документов доверенностей с правом передоверия.
В связи с этим ФНС подготовила обновленный формат электронной доверенности налогоплательщ

In [72]:
data.to_csv("data.csv", index=False)

In [76]:
data['date'] = pd.to_datetime(data['date'])

In [130]:
tags_tokens = data[(data['clusters'] == 7) & (data['date'] < '2022-10-09') & (data['date'] > '2022-09-09')]['tags_tokens']

In [131]:
cv = CountVectorizer() 
word_count_vector = cv.fit_transform(tags_tokens)
tfidf_transformer = TfidfTransformer(smooth_idf=True,use_idf=True) 
tfidf_transformer.fit(word_count_vector)

count_vector = cv.transform(tags_tokens) 
tf_idf_vector = tfidf_transformer.transform(count_vector)
feature_names = cv.get_feature_names()

In [132]:
tfidf_vectors = list()
for i in range(len(tags_tokens)):
    doc_vec = tf_idf_vector[i]
    df = pd.DataFrame(doc_vec.T.todense(), index=feature_names, columns=["tfidf"])
    
    tfidf_vec = list()
    for w in tags_tokens.iloc[i].split():
        try:
            tfidf_vec.append(df.loc[w]['tfidf'])
        except KeyError:
            tfidf_vec.append(0)
    
    tfidf_vectors.append(tfidf_vec)

tfidf = np.array(tfidf_vectors)

In [149]:
a = np.argpartition(tfidf[3], -4)[-4:]
a

array([8, 2, 1, 7])

In [154]:
tags_tokens.tolist()[8]

'расчёт ндфл имущественный налог уплата налог земельный налог транспортный налог налог имущество физический лицо уплата зачёт возврат налог налог доход физический лицо ндфл'

In [137]:
tags_tokens.tolist()[0].split()[8]

'налог'

In [142]:
a = np.argpartition(tfidf[2], -4)[-4:]
a

array([ 6,  7, 10,  1])

In [143]:
tags_tokens.tolist()[2]

'код налоговый льгота имущественный налог земельный налог налог имущество организация налоговый отчётность'

In [148]:
tags_tokens.tolist()[2].split()[1]

'налоговый'